# 1、ConversationTokenBufferMemory的使用

举例1：

In [1]:
import os
import dotenv
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("LLM_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("LLM_BASE_URL")

# 创建大模型实例
llm = ChatOpenAI(model="gpt-4o-mini")

In [2]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.messages import trim_messages

# ====================================================
# 1. 定义离线计数器 (防止网络卡顿的关键)
# ====================================================
def offline_counter(messages) -> int:
    """用字数模拟 Token 数，每 1 个字算 1 个 Token (粗略估算)"""
    total = 0
    for msg in messages:
        total += len(msg.content)
    return total

# ====================================================
# 2. 定义修剪器 (替代 ConversationTokenBufferMemory)
# ====================================================
# 对应旧版的 max_token_limit=10
trimmer = trim_messages(
    max_tokens=10,          # 限制大小
    strategy="last",        # 保留最新的
    token_counter=offline_counter, # 使用离线计数，秒运行
    include_system=True,
    allow_partial=False,    # 不切断单句话
)

# ====================================================
# 3. 操作流程
# ====================================================

# A. 存储消息 (使用 InMemoryChatMessageHistory)
history = InMemoryChatMessageHistory()
history.add_user_message("你好吗？")       # 4个字
history.add_ai_message("我很好，谢谢！")   # 6个字 (累计10)
history.add_user_message("今天天气如何？") # 6个字 (累计16)
history.add_ai_message("晴天，25度")       # 5个字 (累计21)

print(f"--- 原始内存 (共 {len(history.messages)} 条消息) ---")
print(history.messages)

# B. 获取修剪后的结果
# 这步操作通常在 Chain 内部自动做，这里手动演示效果
pruned_messages = trimmer.invoke(history.messages)

print(f"\n--- 优化后：修剪结果 (Max=10) ---")
# 因为最后一条 "晴天，25度" 占5个token，上一条 "今天天气..." 占6个
# 5+6=11 > 10，所以只能保留最后一条
print(pruned_messages)

--- 原始内存 (共 4 条消息) ---
[HumanMessage(content='你好吗？', additional_kwargs={}, response_metadata={}), AIMessage(content='我很好，谢谢！', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]), HumanMessage(content='今天天气如何？', additional_kwargs={}, response_metadata={}), AIMessage(content='晴天，25度', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])]

--- 优化后：修剪结果 (Max=10) ---
[AIMessage(content='晴天，25度', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])]


举例2：

In [3]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.messages import trim_messages

# ====================================================
# 1. 定义离线计数器 (关键优化)
# ====================================================
# 为了防止 tiktoken 下载卡死，用“字数”粗略模拟 Token 数
# 在中文语境下，1个汉字通常对应 0.7~2 个 Token，这里简单按 1:1 计算用于演示
def offline_token_counter(messages) -> int:
    total = 0
    for msg in messages:
        total += len(msg.content)
    return total

# ====================================================
# 2. 定义修剪器 (替代 ConversationTokenBufferMemory)
# ====================================================
# 对应你原来的 max_token_limit=20
trimmer = trim_messages(
    max_tokens=20,          # 设置上限 (这里按字数算)
    strategy="last",        # 保留最新的消息
    token_counter=offline_token_counter, # 使用离线计数
    include_system=True,
    allow_partial=False,    # 不截断单条消息
)

# ====================================================
# 3. 操作流程
# ====================================================

# A. 存储消息 (使用 InMemoryChatMessageHistory)
# 这是一个全量的笔记本，记录所有历史，不会自动删除
history = InMemoryChatMessageHistory()

# Round 1
history.add_user_message("你好吗？")       # 4 字
history.add_ai_message("我很好，谢谢！")   # 6 字

# Round 2
history.add_user_message("今天天气如何？") # 6 字
history.add_ai_message("晴天，25度")       # 5 字

# B. 查看原始记忆
print(f"--- 原始全量记忆 (共 {len(history.messages)} 条) ---")
print(history.messages)

# C. 执行修剪 (模拟 load_memory_variables)
# 这一步通常在 Chain 内部做，这里手动展示效果
# 计算逻辑：
# 最后一条(5) + 倒数第二条(6) + 倒数第三条(6) = 17 < 20
# 如果再加倒数第四条(4) = 21 > 20，超标了！
# 所以预期结果：丢弃最早的 "你好吗？"，保留后三条 (或者根据 trim 策略保留成对)
pruned_messages = trimmer.invoke(history.messages)

print(f"\n--- 优化后：修剪结果 (Max=20) ---")
print(pruned_messages)

--- 原始全量记忆 (共 4 条) ---
[HumanMessage(content='你好吗？', additional_kwargs={}, response_metadata={}), AIMessage(content='我很好，谢谢！', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]), HumanMessage(content='今天天气如何？', additional_kwargs={}, response_metadata={}), AIMessage(content='晴天，25度', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])]

--- 优化后：修剪结果 (Max=20) ---
[AIMessage(content='我很好，谢谢！', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]), HumanMessage(content='今天天气如何？', additional_kwargs={}, response_metadata={}), AIMessage(content='晴天，25度', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])]


# 2、SummaryMemory的使用

举例1：

如果实例化SummaryMemory前，没有历史消息，可以使用构造方法实例化

In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.output_parsers import StrOutputParser

# 1. 创建大模型
llm = ChatOpenAI(model=os.getenv("LLM_MODEL_ID"))

# ==========================================================
# 2. 准备历史数据
# ==========================================================
# 先创建一个历史记录对象，把对话存进去
history = InMemoryChatMessageHistory()
history.add_user_message("你好")
history.add_ai_message("怎么了")
history.add_user_message("你是谁")
history.add_ai_message("我是AI助手小智")
history.add_user_message("初次对话，你能介绍一下你自己吗？")
history.add_ai_message("当然可以了。我是一个无所不能的小智。")

print(f"--- 原始对话条数: {len(history.messages)} ---")

# ==========================================================
# 3. 定义总结链 (替代 ConversationSummaryMemory 的黑盒逻辑)
# ==========================================================
# 这是一个专门用来做“阅读理解”的 Prompt
summary_prompt = ChatPromptTemplate.from_template(
    """
    请将以下的对话内容总结为一个简短的摘要（Summary）。
    摘要应包含关键信息，去除冗余的寒暄。
    
    对话内容：
    {chat_history}
    
    摘要：
    """
)

# 格式化Prompt -> 调用LLM -> 解析成字符串
summarize_chain = summary_prompt | llm | StrOutputParser()

# ==========================================================
# 4. 执行总结
# ==========================================================
# history.messages 转换成字符串传给 chain
# (注：如果模型支持直接读 Message 对象，也可以用 MessagesPlaceholder)
chat_history_str = "\n".join([f"{msg.type}: {msg.content}" for msg in history.messages])

summary = summarize_chain.invoke({"chat_history": chat_history_str})

print("\n--- 优化后：生成的摘要 ---")
print(summary)

--- 原始对话条数: 6 ---

--- 优化后：生成的摘要 ---
人类询问AI的身份，AI介绍自己为无所不能的AI助手小智。


举例2：如果SummaryMemory前，已经有历史消息，可以调用from_messages()实例化

In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.output_parsers import StrOutputParser


# 1. 创建大模型
llm = ChatOpenAI(model=os.getenv("LLM_MODEL_ID"))

# =========================================================
# 2. 准备历史消息 (替代 ChatMessageHistory)
# =========================================================
# 使用 Core 组件 InMemoryChatMessageHistory
history = InMemoryChatMessageHistory()

# 假设的原始消息
history.add_user_message("你好，你是谁？")
history.add_ai_message("我是AI助手小智")

print(f"--- 初始历史消息数: {len(history.messages)} ---")

# =========================================================
# 3. 定义总结链
# =========================================================
# 可以清楚地定义如何做总结，而不是依赖内部写死的英文 Prompt
summary_prompt = ChatPromptTemplate.from_template(
    """
    请阅读以下的对话历史，并将其压缩为一个简练的摘要（Summary）。
    摘要应包含关键信息，如用户身份、AI身份等。

    对话历史：
    {history_str}

    摘要：
    """
)

# 构建 LCEL 链：格式化 -> LLM -> 解析字符串
summarizer_chain = summary_prompt | llm | StrOutputParser()

# 辅助函数：将消息对象转为字符串，并生成摘要
def get_current_summary(chat_history):
    # 将 Message 对象列表转换为文本字符串
    history_str = "\n".join([f"{msg.type}: {msg.content}" for msg in chat_history.messages])
    # 调用链生成摘要
    return summarizer_chain.invoke({"history_str": history_str})

# =========================================================
# 4. 操作流程演示
# =========================================================

# A. 打印初始摘要
print("\n[操作 1] 生成初始摘要...")
summary_v1 = get_current_summary(history)
print(f"摘要内容: {summary_v1}")

# B. 添加新对话
print("\n[操作 2] 添加新对话...")
history.add_user_message("我的名字叫小明")
history.add_ai_message("很高兴认识你")

# C. 再次生成摘要
print("\n[操作 3] 生成更新后的摘要...")
summary_v2 = get_current_summary(history)
print(f"摘要内容: {summary_v2}")

# D. 查看原始消息
print("\n[操作 4] 查看完整历史记录...")
print(history.messages)

--- 初始历史消息数: 2 ---

[操作 1] 生成初始摘要...
摘要内容: 用户询问AI的身份，AI回应称自己是AI助手小智。

[操作 2] 添加新对话...

[操作 3] 生成更新后的摘要...
摘要内容: 用户小明向AI助手小智问好并介绍自己，小智回应并表达欢迎。对话中包含用户身份为小明，AI身份为小智。

[操作 4] 查看完整历史记录...
[HumanMessage(content='你好，你是谁？', additional_kwargs={}, response_metadata={}), AIMessage(content='我是AI助手小智', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]), HumanMessage(content='我的名字叫小明', additional_kwargs={}, response_metadata={}), AIMessage(content='很高兴认识你', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])]
